In [1]:
import wandb
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

import json 
import torch
print(f"Using GPU: {torch.cuda.get_device_name(0)}")

import pandas as pd

from handsoncv.datasets import CILPFusionDataset
from handsoncv.models import LateFusionNet, IntermediateFusionNet
from handsoncv.training import train_fusion_model
from torchvision import transforms
from torch.utils.data import DataLoader

ROOT_PATH = "~/Documents/repos/Applied-Hands-On-Computer-Vision/Assignment-2/"
MOUNTED_ROOT_PATH = os.path.expanduser(ROOT_PATH)
ROOT_DATA = "~/Documents/repos/BuildingAIAgentsWithMultimodalModels/data/assessment/"
IMG_SIZE = 64
BATCH_SIZE = 32

Using GPU: NVIDIA GeForce RTX 3090


In [2]:
# Load split dictionary previouslu created with 01_dataset_exploration.ipynb
mapping_file = "subset_splits.json"
with open(f"{MOUNTED_ROOT_PATH}/{mapping_file}", "r") as f:
    splits = json.load(f)

# Instantiate Dataset
img_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),  # Scales data into [0,1]
])

train_ds = CILPFusionDataset(root_dir=ROOT_DATA, sample_ids=splits["train"], transform=img_transforms)
val_ds = CILPFusionDataset(root_dir=ROOT_DATA, sample_ids=splits["val"], transform=img_transforms)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

print(f"Ready to train with {len(train_ds)} training pairs and {len(val_ds)} validation pairs.")

Ready to train with 3733 training pairs and 3733 validation pairs.


In [3]:
###################################################################
# Sanity Check - Ensure no data leakage between train and val sets
###################################################################

assert set(train_ds.sample_ids).isdisjoint(set(val_ds.sample_ids)), "DATA LEAKAGE DETECTED!"

leaked_ids = set(train_ds.sample_ids).intersection(set(val_ds.sample_ids))
print(f"Found {len(leaked_ids)} overlapping IDs.")
print(f"Example leaked IDs: {list(leaked_ids)[:10]}")

Found 0 overlapping IDs.
Example leaked IDs: []


In [4]:
# Configuration 
EPOCHS = 20
SUBSET_SIZE = len(train_ds) + len(val_ds) # This fulfills the logging requirement
SEED = 42

# Define Experiment Suite
strategies = [
    ("Late Fusion", LateFusionNet(emb_dim=2)),
    ("Int Fusion Concat", IntermediateFusionNet(mode='concat')),
    ("Int Fusion Add", IntermediateFusionNet(mode='add')),
    ("Int Fusion Mul", IntermediateFusionNet(mode='mul')),
]

results = []

for name, model in strategies:
    run = wandb.init(
        project="handsoncv-fusion", 
        name=name,
        config={
            "architecture": name,
            "epochs": EPOCHS,
            "subset_size": SUBSET_SIZE,
            "learning_rate": 1e-4,
            "seed": SEED
        }
    )
    print(f"Training {name}...")
    
    metrics = train_fusion_model(
        model, 
        train_loader, 
        val_loader, 
        optimizer=torch.optim.Adam(model.parameters(), lr=1e-4),
        criterion=torch.nn.CrossEntropyLoss(),
        device="cuda" if torch.cuda.is_available() else "cpu",
        epochs=EPOCHS
    )
    
    metrics['Architecture'] = name
    results.append(metrics)
    wandb.finish()

# --- Final Comparison Table (Task 3.4) ---
# Create DataFrame and reorder columns to match assignment table
df = pd.DataFrame(results)
cols = ["Architecture", "val_loss", "accuracy", "params", "sec_per_epoch", "gpu_mem_mb"]
comparison_table = df[cols]

# Display the table
print("\n" + "="*60)
print("FINAL FUSION COMPARISON TABLE")
print("="*60)
print(comparison_table.to_string(index=False))

wandb: Currently logged in as: guarino-vanessa-emanuela (handsoncv-research) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Training Late Fusion...
Epoch 0: Val Loss: 0.0000, Acc: 100.00% | Mem: 252.1MB
Epoch 1: Val Loss: 0.0000, Acc: 100.00% | Mem: 252.1MB
Epoch 2: Val Loss: 0.0000, Acc: 100.00% | Mem: 252.1MB
Epoch 3: Val Loss: 0.0000, Acc: 100.00% | Mem: 252.1MB
Epoch 4: Val Loss: 0.0000, Acc: 100.00% | Mem: 252.1MB


KeyboardInterrupt: 

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7bbda49d4fd0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7bbf6ef9ca10, execution_count=4 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 7bbda3aa1090, raw_cell="# Configuration 
EPOCHS = 20
SUBSET_SIZE = len(tra.." transformed_cell="# Configuration 
EPOCHS = 20
SUBSET_SIZE = len(tra.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bworkstation/home/vanessa/Documents/repos/Applied-Hands-On-Computer-Vision/Assignment-2/notebooks/02_fusion_exploration.ipynb#W2sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: Connection lost